In [2]:
import pandas as pd
import re
from datetime import datetime

#ya que se necesitará una columna para agregar el resultado del partido, se le hará scrapeo a transfermarket para complementar
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

WEBSCRAPING

In [2]:
#realizamos un escrapeo de la página en que se encuentran las estadisticas
ronaldo = pd.read_html('https://fbref.com/en/players/dea698d9/goallogs/all_comps/Cristiano-Ronaldo-Goal-Log')[0]

In [3]:
#verificamos los datos
ronaldo.head() 

,Rk,Date,Comp,Round,Venue,Squad,Opponent,Start,xG,PSxG,...,Distance,Minute,Score,Goalkeeper,Assist,GCA1,Type,GCA2,Type.1,Notes
0,1.0,2024-10-18,Saudi Professional League,Matchweek 7,Away,Al-Nassr,Al-Shabab,Y*,NaN,NaN,...,NaN,90+7,1-1,Kim Seung-gyu,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,2024-10-12,UEFA Nations League,League A,Away,Portugal,Poland,Y*,NaN,NaN,...,NaN,37,1-0,Łukasz Skorupski,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,2024-10-05,Saudi Professional League,Matchweek 6,Home,Al-Nassr,Al-Orobah,Y*,NaN,NaN,...,NaN,17,0-0,Gaëtan Coucke,NaN,NaN,NaN,NaN,NaN,NaN


Aquí se realizó la extracción para los clubes

In [4]:
# Pasamos la URL a una variable para facilitar su uso
url = 'https://www.transfermarkt.com/cristiano-ronaldo/alletore/spieler/8198/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/minute/0/torart/0/plus/1'

# Agregamos el useragent
user = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=user)
soup = BeautifulSoup(response.content, "html.parser")

# Guardamos en una variable los selectores que ocupamos (esto se puede encontrar al inspeccionar la página a scrapear)
table_div = soup.select_one('div[class="responsive-table"]')

# Verificamos si se encontró el div
if table_div is None:
    print("no se encontró el div con la clase 'responsive-table'")
else:
    # Seleccionamos la tabla dentro del div
    table = table_div.find('table')
    print(table)

    # Verificamos si se encontró la tabla
    if table is None:
        print('No se encontró la tabla dentro del div')
    else:
        # Extraemos los headers
        headers = [header.get_text(strip=True) for header in table.find('thead').find_all('th')]
        # Añadimos un nuevo header para la columna de resultado (WIN/LOSS/DRAW)
        headers.append('Match Result')
        print('Headers: ', headers)

        # Ajustamos el scraping para colocar la columna 'Match Result' en la posición correcta
        rows = []

        # Agregamos un bloque para ajustar las filas que tienen un desajuste en el número de columnas
for row in table.find('tbody').find_all('tr'):
    if row.find('td', {'colspan': '18'}):
        continue

    cells = row.find_all('td')
    cells_data = []

    for cell in cells:
        # Omitimos celdas con imágenes
        if cell.find('img') is not None:
            continue
        # Si la celda tiene 'colspan=2', insertamos el título en el lugar que corresponde
        if cell.has_attr('colspan') and cell['colspan'] == '2':
            title = cell.get('title')  # Obtenemos el título si está presente
            if title:
                cells_data.append(title)  # Añadimos el título
                cells_data.append('')  # Añadimos una celda vacía adicional
            else:
                cells_data.append('')
        else:
            cells_data.append(cell.get_text(strip=True))

    # Inicializamos una variable para almacenar el resultado del partido
    match_result = 'Unknown'

    # Buscamos si en alguna celda está el texto con 'greentext' o 'redtext'
    for cell in cells:
        span_green = cell.find('span', class_='greentext')
        span_red = cell.find('span', class_='redtext')
        if span_green:
            match_result = 'Win'
        elif span_red:
            match_result = 'Loss'

    # Si el resultado sigue siendo 'Unknown', verificamos si es un empate
    if match_result == 'Unknown':
        for cell in cells:
            result_text = cell.get_text(strip=True)
            if ':' in result_text:
                scores = result_text.split(':')
                if len(scores) == 2 and scores[0] == scores[1]:
                    match_result = 'Draw'

    # Ajustamos la cantidad de celdas si es menor a la cantidad de headers
    if len(cells_data) < len(headers) - 1:
        while len(cells_data) < len(headers) - 1:
            cells_data.insert(-2, '')  # Insertamos celdas vacías hasta que coincidan con los headers

    # Añadimos el resultado del partido como la última columna
    cells_data.append(match_result)

    # Verificamos si el número de celdas coincide con los headers
    if len(cells_data) == len(headers):
        rows.append(cells_data)
    else:
        print(f'Fila ignorada debido a desajuste de columnas: {cells_data}')

# Convertimos a DataFrame
if rows:
    transfermkt_df = pd.DataFrame(rows, columns=headers)
    print(transfermkt_df)
else:
    print('No se encontraron filas en la tabla')


<table>
<thead>
<tr>
<th class="" colspan="2">Competition</th>
<th class="zentriert">Matchday</th>
<th class="zentriert">Date</th>
<th class="zentriert">Venue</th>
<th class="zentriert" colspan="2">For</th>
<th colspan="2">Opponent</th>
<th class="zentriert">Result</th>
<th class="zentriert">Pos.</th>
<th class="zentriert">Minute</th>
<th class="zentriert">At score</th>
<th class="">Type of goal</th>
<th class="">Goal assist</th>
</tr>
</thead>
<tbody>
<tr style="border-top:1px dotted red  !important;">
<td class="hauptlink highlight" colspan="18">Season 02/03</td>
</tr>
<tr class="">
<td class="zentriert no-border-rechts">
<img alt="Liga Portugal" class="" src="https://tmssl.akamaized.net//images/logo/tiny/po1.png?lm=1688630708" title="Liga Portugal"> </img></td>
<td class="links no-border-links">
<a href="/liga-nos/startseite/wettbewerb/PO1" title="Liga Portugal">Liga Portugal</a> </td>
<td class="zentriert">
<a href="/liga-portugal/spieltag/wettbewerb/PO1/saison_id/2002/spieltag/6">

In [5]:
transfermkt_df.head()

,Competition,Matchday,Date,Venue,For,Opponent,Result,Pos.,Minute,At score,Type of goal,Goal assist,Match Result
0,Liga Portugal,6,10/7/02,H,(12.),Moreirense(16.),3:0,LW,34',2:0,Solo run,,Win
1,90'+53:0HeaderRui Jorge,90'+5,3:0,,,,,,,,Header,Rui Jorge,Unknown
2,Liga Portugal,8,10/26/02,A,(5.),Boavista(10.),1:2,,88',1:2,Right-footed shot,Carlos Martins,Win
3,Taça de Portugal,4.Runde,11/24/02,H,Estarreja,4:1,LW,67',3:1,,Left-footed shot,César Prates,Win
4,Taça de Portugal,5.Runde,12/18/02,H,Oliv. Hospital,8:1,SS,13',3:0,,Left-footed shot,Ricardo Fernandes,Win


In [6]:
transfermkt_df.shape

(768, 13)

Scrapeamos los traofeos de igual forma

In [3]:
#ahora buscaremos los resultados para los trofeos
url = 'https://www.transfermarkt.com/cristiano-ronaldo/erfolge/spieler/8198'

#agregamos el useragent, el cual se puede verificar en la siguiente página web https://www.whatismybrowser.com/detect/what-http-headers-is-my-browser-sending/
user = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=user)

response.status_code
soup = BeautifulSoup(response.content, "html.parser")
#verificamos las tablas que se encuentran 
tables = soup.find_all('table')
#imprimimos cuantas tablas se encontraron
print(f'se encontraron {len(tables)} tablas')

#si se enontraron varias tablas, podemos inspeccionarlas para conocer qué tabla necesitamos del codigo
for i, table in enumerate(tables):
    print(f'Tabla {i}')
    print(table.prettify())

se encontraron 26 tablas
Tabla 0
<table class="auflistung">
 <tr>
  <td class="erfolg_table_saison zentriert">
   2017
  </td>
  <td class="zentriert no-border-rechts" style="width:30px;">
   <a href="/real-madrid/startseite/verein/418/saison_id/2016" title="Real Madrid">
    <img alt="Real Madrid" class="tiny_wappen" src="https://tmssl.akamaized.net//images/wappen/tiny/418.png?lm=1729684474" title="Real Madrid"/>
   </a>
  </td>
  <td class="no-border-links">
   <a href="/real-madrid/startseite/verein/418/saison_id/2016" title="Real Madrid">
    Real Madrid
   </a>
  </td>
 </tr>
 <tr>
  <td class="erfolg_table_saison zentriert">
   2016
  </td>
  <td class="zentriert no-border-rechts" style="width:30px;">
   <a href="/real-madrid/startseite/verein/418/saison_id/2015" title="Real Madrid">
    <img alt="Real Madrid" class="tiny_wappen" src="https://tmssl.akamaized.net//images/wappen/tiny/418.png?lm=1729684474" title="Real Madrid"/>
   </a>
  </td>
  <td class="no-border-links">
   <a h

In [4]:
#buscamos la tabla a usar y la guardamos en una variable
if len(tables) >= 25:
    table25 = tables[25] #tabla 25 en la lista
    #inicializamos la lista para almacenar
    data = []

    #encontramos cada encabezado del premio y sus detalles dentro de la tabla seleccionada
    for header in table25.find_all('tr', class_='bg_Sturm'):
        award_name = header.text.strip()

        #removemos los nmeros seguidos de "x" así como la x y los espacios posterior a esta
        award_name = re.sub(r"^\d+x\s*","", award_name)

        #siguiente fila para encontrar los datos asociados
        next_row = header.find_next_sibling('tr')

        #recorremos las filas hasta encontrar otro encabezado o el final
        while next_row and 'bg_Sturm' not in next_row.get("class",[]):
            #extraemos el año y el título
            year_cell = next_row.find("td",class_="erfolg_table_saison zentriert")
            title_cell = next_row.find("td", class_='no-border-links')

            #verificamos que ambos elementos existen antes de acceder al texto
            if year_cell and title_cell:
                year = year_cell.text.strip()
                title = title_cell.text.strip()

                #agregar a la lista
                data.append({'Award': award_name, "Year": year, "Squad":title})

            #movemos a la siguiente fila
            next_row = next_row.find_next_sibling('tr')

    #creamos el nuevo DataFrame
    awards = pd.DataFrame(data)

    #verificamos el resultado
    print("seleccionamos la tabla correcta:")
    print(awards)
else:
    print("no se encontró la tabla numero 25 dentro del HTML")

seleccionamos la tabla correcta:
                          Award   Year              Squad
0             European champion   2016           Portugal
1    The Best FIFA Men's Player   2017        Real Madrid
2    The Best FIFA Men's Player   2016        Real Madrid
3    The Best FIFA Men's Player   2008  Manchester United
4    UEFA Best Player in Europe   2017        Real Madrid
..                          ...    ...                ...
143          Player of the Year  07/08  Manchester United
144          Player of the Year  06/07  Manchester United
145     TM-Player of the season   2024       Saudi Arabia
146     TM-Player of the season   2019              Italy
147     TM-Player of the season   2017              Spain

[148 rows x 3 columns]


In [5]:
#eliminaremos las filas en que la columna award tenga paticipant en sus filas, ya que estos solo son premios por participar y no por ganar
awards = awards[~awards['Award'].str.contains('participant', case=False, na=False)]
#Eliminamos los goles agregados en la columna squad
awards.loc[:,'Squad'] = awards['Squad'].str.split('-').str[0].str.strip()
#ya que al hacer la eliminación de los goles se va el nombre de al nassr en una fila habrá que modificarlo
awards.loc[awards['Squad'] == "Al", 'Squad'] = 'Al-Nassr'
#restablecemos los indices del DF
awards.reset_index(drop=True,inplace=True)
#verificamos los resultados
awards

,Award,Year,Squad
0,European champion,2016,Portugal
1,The Best FIFA Men's Player,2017,Real Madrid
2,The Best FIFA Men's Player,2016,Real Madrid
3,The Best FIFA Men's Player,2008,Manchester United
4,UEFA Best Player in Europe,2017,Real Madrid
...,...,...,...
99,Player of the Year,07/08,Manchester United
100,Player of the Year,06/07,Manchester United
101,TM-Player of the season,2024,Saudi Arabia
102,TM-Player of the season,2019,Italy


In [11]:
#guardamos el df 
awards.to_csv("D:\\RonaldoStats\\Data\\awards.csv", encoding="utf-8")

Aquí se realizará la extracción del resultado para el equipo nacional

In [7]:
#ahora buscaremos los resultados para el equipo nacional
url = 'https://www.transfermarkt.com/cristiano-ronaldo/nationalmannschaft/spieler/8198/verein_id/3300/hauptwettbewerb//wettbewerb_id//start/2003-08-20/ende/2024-10-19/nurEinsatz/0'

#agregamos el useragent, el cual se puede verificar en la siguiente página web https://www.whatismybrowser.com/detect/what-http-headers-is-my-browser-sending/
user = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=user)

response.status_code
soup = BeautifulSoup(response.content, "html.parser")


In [8]:
#guardamos todas las tablas que se encuentren en el código, para posteriormente identificar la que necesitaremos utilizar
tables = soup.find_all('table')

#imprimimos cuantas tablas se encontraron
print(f'se encontraron {len(tables)} tablas')

#si se encontraron varias tablas podemos inspeccionarlas para ver cuál ocupamos
for i, table in enumerate(tables):
    print(f'Tabla {i}')
    print(table.prettify())
    

se encontraron 5 tablas
Tabla 0
<table>
 <thead>
  <tr>
   <th class="zentriert">
    #
   </th>
   <th class="hide-for-small" colspan="2">
    National team
   </th>
   <th class="zentriert">
    Debut
   </th>
   <th class="zentriert">
    <span class="icons_sprite icon-einsaetze-table-header" title="Matches">
    </span>
   </th>
   <th class="zentriert">
    <span class="icons_sprite icon-tor-table-header">
    </span>
   </th>
   <th>
    Coach at debut
   </th>
   <th class="rechts">
    Age at debut
   </th>
  </tr>
 </thead>
 <tbody>
  <tr>
   <td class="no-border-rechts zentriert ueberzeile show-for-small">
    <img alt="Portugal" class="flaggenrahmen" src="https://tmssl.akamaized.net//images/flagge/verysmall/136.png?lm=1520611569" title="Portugal">
    </img>
   </td>
   <td class="hauptlink no-border-links ueberzeile show-for-small" colspan="5">
    <a href="/portugal/startseite/verein/3300" title="Portugal">
     Portugal
    </a>
    <span class="kapitaenicon-table icons_s

In [9]:
#aquí se realizará el código para obtener el df de las stadisticas para el equipo nacional

#cómo vimos en el código anterior necesitamos la tabla 4
if len(tables) >=4:
    table4 = tables[4] #seleccionamos la tabla 4 en la lista
    print('Se seleccionó la tabla correcta: ')
    print(table4)
    #extraemos los heders
    headers= []

    for i, header in enumerate(table4.find('thead').find_all('th')):
        if i == 0: #saltamos el primer th porque está vacío
            continue
        span = header.find('span')
        
        if span and 'title' in span.attrs:
            #si existe el span y tiene atributo extraemos el 
            headers.append(span['title'])
        else:
            #si no hay span, extraemos el  del th
            headers.append(header.get_text(strip=True))

    #añadimos un nuevo heder para la columna de resultado (WIn/Loss/Draw)
    headers.append('Match Result')
    #verificamos los heders
    print(f'Headers: {headers}')

    #obtenemos todas las filas de la tabla dentro  de tbody
    rows = []
    for row in table4.find('tbody').find_all('tr'):
        #ignoramos las filas que tienen colspan 20 y 8, ya que no contienen datos correspondientes a los headers
        if row.find('td',{'colspan':'20'}) or row.find('td',{'colspan':'8'}):
            continue
        
        #extraemos los datos de cada celda <td> y omitimos celdas con imágenes
        cells = row.find_all('td')
        
        #saltamos la primera celda vacía (correspondiente al th vacío)
        cells = cells[1:]

        #procesamos las celdas, si hay imagenes ponemos None
        cells_data =[]
        for cell in cells:
            img = cell.find('img')
            if img and img.get('title') == 'Portugal':
                #si encontramos una imagen y el titulo es portugal, agregamos el título
                cells_data.append('Portugal')
            elif img is None:
                #si no hay imagen, agregamos el texto de la celda
                cells_data.append(cell.get_text(strip=True))
        #inicializamos una variable para almacenar los resultados del partido
        match_result = 'Unknown'

        #buscamos si en alguna celda se encuentra el texto con el greentext o redtext
        for cell in cells:
            span_green = cell.find('span', class_='greentext')
            span_red = cell.find('span', class_='redtext')
            if span_green:
                match_result = 'Win' #si es greentext, es una victoria
            elif span_red:
                match_result = 'Loss' #si es redtext, es una derrota
        
        #si el resultado sigue siendo 'Unkown', verificamos si es un empate
        #los resultados tienen el formato "X:X", así que bsucamos esos casos
        if match_result == 'Unknown':
            for cell in cells:
                result_text = cell.get_text(strip=True)
                if ':' in result_text: #comprobamos si el texto contiene un marcador
                    scores = result_text.split(':')
                    if len(scores) == 2 and scores[0] == scores[1]: #verificamos si se complen estas condiciones para decir que es empate
                        match_result = 'Draw'
        
        #verificamos si el número de celdas coincide con el número de headers ( sin contar el nuevo)
        if len(cells_data) == len(headers) -1:
            cells_data.append(match_result) #añadimos el resultado del partido
            rows.append(cells_data)
        else:
            print(f'Fila ignorada debido a desajusto de columnas{cells_data}')
    
    #imprimimos todas las filas capturadas
    print(f'Todas las filas: {rows}')

    #creamos el Dataframe
    if rows:
        transfermkt_Nacionl = pd.DataFrame(rows, columns=headers)
    else:
        print('No se encontraron filas en la tabla')

transfermkt_Nacionl.head(10)        

Se seleccionó la tabla correcta: 
<table>
<thead>
<tr>
<th class="zentriert"> </th>
<th class="zentriert">Matchday</th>
<th class="zentriert">Date</th>
<th class="zentriert">Venue</th>
<th class="zentriert" colspan="2">For</th>
<th colspan="2">Opponent</th>
<th class="zentriert">Result</th>
<th class="zentriert">Pos.</th>
<th class="zentriert"><span class="icons_sprite icon-tor-table-header" title="Goals"> </span></th>
<th class="zentriert"><span class="icons_sprite icon-vorlage-table-header" title="Assists"> </span></th>
<th class="zentriert"><span class="icons_sprite icon-gelbekarte-table-header" title="Yellow cards"> </span></th>
<th class="zentriert"><span class="icons_sprite icon-gelbrotekarte-table-header" title="Second yellow cards"> </span></th>
<th class="zentriert"><span class="icons_sprite icon-rotekarte-table-header" title="Red cards"> </span></th>
<th class="rechts"><span class="icons_sprite icon-minuten-table-header" title="Minutes played"> </span></th>
</tr>
</thead>
<tb

,Matchday,Date,Venue,For,Opponent,Result,Pos.,Goals,Assists,Yellow cards,Second yellow cards,Red cards,Minutes played,Match Result
0,,8/20/03,H,Portugal,Kazakhstan,1:0,RW,,,,,,45',Win
1,,10/11/03,H,Portugal,Albania,5:3,RW,,,,,,45',Win
2,,2/18/04,H,Portugal,England,1:1,LW,,,,,,45',Draw
3,,3/31/04,H,Portugal,Italy,1:2,LW,,,,,,45',Loss
4,,4/28/04,H,Portugal,Sweden,2:2,RW,,1,,,,27',Draw
5,,5/29/04,H,Portugal,Luxembourg,3:0,RW,,,,,,45',Win
6,,6/5/04,H,Portugal,Lithuania,4:1,RW,,,,,,26',Win
7,Group A,6/12/04,H,Portugal,Greece,1:2,LW,1,,,,,45',Loss
8,Group A,6/16/04,H,Portugal,Russia,0:2,LW,,1,,,,12',Win
9,Group A,6/20/04,H,Portugal,Spain,0:1,LW,,,,,,85',Win


Cambiar Penalti por Penalty

En el df Transfermkt_df y Nacional mantenemos unicamente el Date y match result ya que el Date nos servirá para relacionar con la tabla que tenemos completa

Mantener unicamente las filas que contengan goal

Dar formato a la columna Date de transfermkt_df y transfermkt_nacional



ETL


In [10]:
#verificamos columnas
ronaldo.columns

Index(['Rk', 'Date', 'Comp', 'Round', 'Venue', 'Squad', 'Opponent', 'Start',
       'xG', 'PSxG', 'Body Part', 'Distance', 'Minute', 'Score', 'Goalkeeper',
       'Assist', 'GCA1', 'Type', 'GCA2', 'Type.1', 'Notes'],
      dtype='object')

In [11]:
#eliminamos columnas que no se usarán
ronaldo.drop(columns=['xG','PSxG', 'GCA1', 'Type', 'GCA2', 'Type.1', 'Notes','Distance'], inplace=True)
ronaldo.head(5)

,Rk,Date,Comp,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist
0,1.0,2024-10-18,Saudi Professional League,Matchweek 7,Away,Al-Nassr,Al-Shabab,Y*,NaN,90+7,1-1,Kim Seung-gyu,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,2024-10-12,UEFA Nations League,League A,Away,Portugal,Poland,Y*,NaN,37,1-0,Łukasz Skorupski,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,2024-10-05,Saudi Professional League,Matchweek 6,Home,Al-Nassr,Al-Orobah,Y*,NaN,17,0-0,Gaëtan Coucke,NaN


In [12]:
#verificamos el tamaño del df
ronaldo.shape

(935, 13)

In [13]:
#verificamos nulos
ronaldo.isna().sum()

Rk            104
Date          104
Comp          104
Round         104
Venue         104
Squad         104
Opponent      104
Start         104
Body Part     762
Minute        104
Score         165
Goalkeeper    239
Assist        583
dtype: int64

In [14]:
#eliminamos las filas que en su totalidad son nulos
ronaldo.dropna(how='all', inplace=True)
#reiniciamos los indices
ronaldo.reset_index(drop=True, inplace=True)

In [15]:
#verificamos nulos
ronaldo.isna().sum()

Rk              0
Date            0
Comp            0
Round           0
Venue           0
Squad           0
Opponent        0
Start           0
Body Part     658
Minute          0
Score          61
Goalkeeper    135
Assist        479
dtype: int64

In [16]:
ronaldo.shape

(831, 13)

In [17]:
ronaldo.head()

,Rk,Date,Comp,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist
0,1.0,2024-10-18,Saudi Professional League,Matchweek 7,Away,Al-Nassr,Al-Shabab,Y*,NaN,90+7,1-1,Kim Seung-gyu,NaN
1,2.0,2024-10-12,UEFA Nations League,League A,Away,Portugal,Poland,Y*,NaN,37,1-0,Łukasz Skorupski,NaN
2,3.0,2024-10-05,Saudi Professional League,Matchweek 6,Home,Al-Nassr,Al-Orobah,Y*,NaN,17,0-0,Gaëtan Coucke,NaN
3,4.0,2024-09-27,Saudi Professional League,Matchweek 5,Home,Al-Nassr,Al-Wehda,Y*,NaN,56,1-0,Abdullah Al-Owaishir,NaN
4,5.0,2024-09-20,Saudi Professional League,Matchweek 4,Away,Al-Nassr,Al-Ettifaq,Y*,NaN,33,0-0,Marek Rodák,NaN


In [18]:
#modificamos el tipo de dato en Rk
ronaldo['Rk'] = ronaldo['Rk'].astype(int)
ronaldo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 831 entries, 0 to 830
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Rk          831 non-null    int32 
 1   Date        831 non-null    object
 2   Comp        831 non-null    object
 3   Round       831 non-null    object
 4   Venue       831 non-null    object
 5   Squad       831 non-null    object
 6   Opponent    831 non-null    object
 7   Start       831 non-null    object
 8   Body Part   173 non-null    object
 9   Minute      831 non-null    object
 10  Score       770 non-null    object
 11  Goalkeeper  696 non-null    object
 12  Assist      352 non-null    object
dtypes: int32(1), object(12)
memory usage: 81.3+ KB


In [19]:
#modificamos los tipos de datos a lo que corresponden
ronaldo['Date'] = pd.to_datetime(ronaldo['Date'])
ronaldo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 831 entries, 0 to 830
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Rk          831 non-null    int32         
 1   Date        831 non-null    datetime64[ns]
 2   Comp        831 non-null    object        
 3   Round       831 non-null    object        
 4   Venue       831 non-null    object        
 5   Squad       831 non-null    object        
 6   Opponent    831 non-null    object        
 7   Start       831 non-null    object        
 8   Body Part   173 non-null    object        
 9   Minute      831 non-null    object        
 10  Score       770 non-null    object        
 11  Goalkeeper  696 non-null    object        
 12  Assist      352 non-null    object        
dtypes: datetime64[ns](1), int32(1), object(11)
memory usage: 81.3+ KB


In [20]:
#llenamos las filas que no contengan datos
ronaldo = ronaldo.fillna('sin dato')

In [21]:
#Verificamos
ronaldo.tail(6)

,Rk,Date,Comp,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist
825,826,2004-04-10,Premier League,Matchweek 32,Away,Manchester Utd,Birmingham City,N,sin dato,60,0-1,Maik Taylor,Ryan Giggs
826,827,2004-03-20,Premier League,Matchweek 29,Home,Manchester Utd,Tottenham,N,sin dato,89,1-0,Kasey Keller,Roy Keane
827,828,2003-11-01,Premier League,Matchweek 11,Home,Manchester Utd,Portsmouth,N,sin dato,80,1-0,Shaka Hislop,sin dato
828,829,2002-10-26,Primeira Liga,Matchweek 8,Away,Sporting CP,Boavista,N,sin dato,88,1-1,sin dato,sin dato
829,830,2002-10-07,Primeira Liga,Matchweek 6,Home,Sporting CP,Moreirense,Y,sin dato,90,2-0,sin dato,sin dato
830,831,2002-10-07,Primeira Liga,Matchweek 6,Home,Sporting CP,Moreirense,Y,sin dato,34,1-0,sin dato,sin dato


In [22]:
#verificamos nulos de nuevo
ronaldo.isna().sum()

Rk            0
Date          0
Comp          0
Round         0
Venue         0
Squad         0
Opponent      0
Start         0
Body Part     0
Minute        0
Score         0
Goalkeeper    0
Assist        0
dtype: int64

In [23]:
ronaldo.loc[ronaldo['Body Part'] == 'sin dato']

,Rk,Date,Comp,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist
0,1,2024-10-18,Saudi Professional League,Matchweek 7,Away,Al-Nassr,Al-Shabab,Y*,sin dato,90+7,1-1,Kim Seung-gyu,sin dato
1,2,2024-10-12,UEFA Nations League,League A,Away,Portugal,Poland,Y*,sin dato,37,1-0,Łukasz Skorupski,sin dato
2,3,2024-10-05,Saudi Professional League,Matchweek 6,Home,Al-Nassr,Al-Orobah,Y*,sin dato,17,0-0,Gaëtan Coucke,sin dato
3,4,2024-09-27,Saudi Professional League,Matchweek 5,Home,Al-Nassr,Al-Wehda,Y*,sin dato,56,1-0,Abdullah Al-Owaishir,sin dato
4,5,2024-09-20,Saudi Professional League,Matchweek 4,Away,Al-Nassr,Al-Ettifaq,Y*,sin dato,33,0-0,Marek Rodák,sin dato
...,...,...,...,...,...,...,...,...,...,...,...,...,...
826,827,2004-03-20,Premier League,Matchweek 29,Home,Manchester Utd,Tottenham,N,sin dato,89,1-0,Kasey Keller,Roy Keane
827,828,2003-11-01,Premier League,Matchweek 11,Home,Manchester Utd,Portsmouth,N,sin dato,80,1-0,Shaka Hislop,sin dato
828,829,2002-10-26,Primeira Liga,Matchweek 8,Away,Sporting CP,Boavista,N,sin dato,88,1-1,sin dato,sin dato
829,830,2002-10-07,Primeira Liga,Matchweek 6,Home,Sporting CP,Moreirense,Y,sin dato,90,2-0,sin dato,sin dato


In [24]:
#guardamos en csv para su modificacion manual en excel, ya que hay que realizar una busqueda de los datos faltantes
ronaldo.to_csv('D:/RonaldoStats/Data/Ronaldo_Rawdata.csv', encoding='utf-8')

In [25]:
#abrimos el csv modificado con los datos faltantes
cr7 = pd.read_csv('D:\\RonaldoStats\\ronaldo.csv', encoding='utf-8')
cr7.head(5)

,Rk,Date,Competition,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist,Dorsal,Edad,Goal
0,1,09/05/2024,Pro League,Matchweek 31,Away,Al-Nassr,Al Okhdood SC,Y*,Right Foot,15,3-2,Paulo Vítor Fagundes dos Anjos,Ali Al Hassan,7,39,1
1,2,04/05/2024,Pro League,Matchweek 30,Home,Al-Nassr,Al-Wehda,Y*,Head,12,6-0,Munir,Marcelo Brozović,7,39,1
2,3,04/05/2024,Pro League,Matchweek 30,Home,Al-Nassr,Al-Wehda,Y*,Left Foot,52,6-0,Munir,Sadio Mané,7,39,1
3,4,04/05/2024,Pro League,Matchweek 30,Home,Al-Nassr,Al-Wehda,Y*,Head,5,6-0,Munir,No assist,7,39,1
4,5,01/05/2024,Kings CUP,Semi-finals,Home,Al-Nassr,Al-Khaleej,Y*,Left Foot,57,3-1,Sehic,Ayman Yahya,7,39,1


In [26]:
cr7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Rk           891 non-null    int64 
 1   Date         891 non-null    object
 2   Competition  891 non-null    object
 3   Round        891 non-null    object
 4   Venue        891 non-null    object
 5   Squad        891 non-null    object
 6   Opponent     891 non-null    object
 7   Start        891 non-null    object
 8   Body Part    891 non-null    object
 9   Minute       891 non-null    object
 10  Score        891 non-null    object
 11  Goalkeeper   891 non-null    object
 12  Assist       891 non-null    object
 13  Dorsal       891 non-null    int64 
 14  Edad         891 non-null    int64 
 15  Goal         891 non-null    int64 
dtypes: int64(4), object(12)
memory usage: 111.5+ KB


In [27]:
#convertimos a datetime con el formato correcto dayfirst
cr7['Date'] = pd.to_datetime(cr7['Date'], dayfirst=True)

#cambiamos el formato a "YYYY-MM-DD" para utilizarlo sin tantas complicaciones
cr7['Date'] = cr7['Date'].dt.strftime('%Y-%m-%d')

cr7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Rk           891 non-null    int64 
 1   Date         891 non-null    object
 2   Competition  891 non-null    object
 3   Round        891 non-null    object
 4   Venue        891 non-null    object
 5   Squad        891 non-null    object
 6   Opponent     891 non-null    object
 7   Start        891 non-null    object
 8   Body Part    891 non-null    object
 9   Minute       891 non-null    object
 10  Score        891 non-null    object
 11  Goalkeeper   891 non-null    object
 12  Assist       891 non-null    object
 13  Dorsal       891 non-null    int64 
 14  Edad         891 non-null    int64 
 15  Goal         891 non-null    int64 
dtypes: int64(4), object(12)
memory usage: 111.5+ KB


In [28]:
cr7['Date'] = pd.to_datetime(cr7['Date'], errors='coerce')

In [29]:
cr7.head()

,Rk,Date,Competition,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist,Dorsal,Edad,Goal
0,1,2024-05-09,Pro League,Matchweek 31,Away,Al-Nassr,Al Okhdood SC,Y*,Right Foot,15,3-2,Paulo Vítor Fagundes dos Anjos,Ali Al Hassan,7,39,1
1,2,2024-05-04,Pro League,Matchweek 30,Home,Al-Nassr,Al-Wehda,Y*,Head,12,6-0,Munir,Marcelo Brozović,7,39,1
2,3,2024-05-04,Pro League,Matchweek 30,Home,Al-Nassr,Al-Wehda,Y*,Left Foot,52,6-0,Munir,Sadio Mané,7,39,1
3,4,2024-05-04,Pro League,Matchweek 30,Home,Al-Nassr,Al-Wehda,Y*,Head,5,6-0,Munir,No assist,7,39,1
4,5,2024-05-01,Kings CUP,Semi-finals,Home,Al-Nassr,Al-Khaleej,Y*,Left Foot,57,3-1,Sehic,Ayman Yahya,7,39,1


Realizamos una nueva columna de 'Edad' al momento del gol. Se realizará basandonos en la fecha de nacimiento y la columna date
Ademas de agregar una columna de Goal 

In [30]:
#función para agregar una columna de Goal, esto con la intención de facilitar su uso al visulizar por medio de PBI o Tableu
def colum_goal(df, column_name, value):
    #añadimos la columna 
    setattr(df,column_name, value)
    df[column_name] = value
    return df

In [31]:
#llamamos a la funcion para añadr la nueva columna
colum_goal(ronaldo, 'Goal',1)
ronaldo.head(5)

,Rk,Date,Comp,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist,Goal
0,1,2024-10-18,Saudi Professional League,Matchweek 7,Away,Al-Nassr,Al-Shabab,Y*,sin dato,90+7,1-1,Kim Seung-gyu,sin dato,1
1,2,2024-10-12,UEFA Nations League,League A,Away,Portugal,Poland,Y*,sin dato,37,1-0,Łukasz Skorupski,sin dato,1
2,3,2024-10-05,Saudi Professional League,Matchweek 6,Home,Al-Nassr,Al-Orobah,Y*,sin dato,17,0-0,Gaëtan Coucke,sin dato,1
3,4,2024-09-27,Saudi Professional League,Matchweek 5,Home,Al-Nassr,Al-Wehda,Y*,sin dato,56,1-0,Abdullah Al-Owaishir,sin dato,1
4,5,2024-09-20,Saudi Professional League,Matchweek 4,Away,Al-Nassr,Al-Ettifaq,Y*,sin dato,33,0-0,Marek Rodák,sin dato,1


In [32]:
#funcion para agregar la columna Edad
def add_age(df,fecha_nacimiento,column_name):
    #convertimos la columna 'Date' a formato de fecha, manejando posibles errores
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

    #calculamos la edad
    df[column_name] = (df['Date'] - fecha_nacimiento).dt.days // 365

    # retornamos el df
    return df


In [33]:
cumple_ronaldo =datetime(1985,2,5) #necesitamos pasar la fecha como datetime, sino lo tomará como tupla y no ser hará el calculo

add_age(ronaldo,cumple_ronaldo,'Edad')
ronaldo.head(5)

,Rk,Date,Comp,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist,Goal,Edad
0,1,2024-10-18,Saudi Professional League,Matchweek 7,Away,Al-Nassr,Al-Shabab,Y*,sin dato,90+7,1-1,Kim Seung-gyu,sin dato,1,39
1,2,2024-10-12,UEFA Nations League,League A,Away,Portugal,Poland,Y*,sin dato,37,1-0,Łukasz Skorupski,sin dato,1,39
2,3,2024-10-05,Saudi Professional League,Matchweek 6,Home,Al-Nassr,Al-Orobah,Y*,sin dato,17,0-0,Gaëtan Coucke,sin dato,1,39
3,4,2024-09-27,Saudi Professional League,Matchweek 5,Home,Al-Nassr,Al-Wehda,Y*,sin dato,56,1-0,Abdullah Al-Owaishir,sin dato,1,39
4,5,2024-09-20,Saudi Professional League,Matchweek 4,Away,Al-Nassr,Al-Ettifaq,Y*,sin dato,33,0-0,Marek Rodák,sin dato,1,39


In [ ]:
"""
Otra forma de agregar sin funciones

#agregamos las columnas faltantes 'Edad', 'Goal' en el DF 'ronaldo' y poder combinar las fechas faltantes
ronaldo_cumple = datetime(1985,2,5)  #5 de febrero de 1985 es la fecha de nacimiento 

#convertimos la columna 'Date' a formato fecha

ronaldo['Date'] = pd.to_datetime(ronaldo['Date'], errors='coerce')

#calculamos la edad en años en el momento del gol, tomamos en cuenta los años bisiestos por si a caso
ronaldo['Edad'] = (ronaldo['Date'] - ronaldo_cumple).dt.days // 365
ronaldo['Goal'] = 7
ronaldo.head(3)
"""

In [34]:
#cambiamos el nombre de la columna Comp por Competition
nm = {'Comp': 'Competition'}
ronaldo.rename(columns=nm, inplace=True)
#verificamos el cambio
ronaldo.head(1)

,Rk,Date,Competition,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist,Goal,Edad
0,1,2024-10-18,Saudi Professional League,Matchweek 7,Away,Al-Nassr,Al-Shabab,Y*,sin dato,90+7,1-1,Kim Seung-gyu,sin dato,1,39


In [35]:
datos = ronaldo.head(13) #guardamos las primeras 13 filas que son las que realmente necesitamos
filas_selecc = pd.DataFrame(datos)

#liberamos espacio
del datos

#agregamos la columna de Dorsal 
filas_selecc['Dorsal'] = 7

#verificamos el DF
filas_selecc.head(13)

,Rk,Date,Competition,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist,Goal,Edad,Dorsal
0,1,2024-10-18,Saudi Professional League,Matchweek 7,Away,Al-Nassr,Al-Shabab,Y*,sin dato,90+7,1-1,Kim Seung-gyu,sin dato,1,39,7
1,2,2024-10-12,UEFA Nations League,League A,Away,Portugal,Poland,Y*,sin dato,37,1-0,Łukasz Skorupski,sin dato,1,39,7
2,3,2024-10-05,Saudi Professional League,Matchweek 6,Home,Al-Nassr,Al-Orobah,Y*,sin dato,17,0-0,Gaëtan Coucke,sin dato,1,39,7
3,4,2024-09-27,Saudi Professional League,Matchweek 5,Home,Al-Nassr,Al-Wehda,Y*,sin dato,56,1-0,Abdullah Al-Owaishir,sin dato,1,39,7
4,5,2024-09-20,Saudi Professional League,Matchweek 4,Away,Al-Nassr,Al-Ettifaq,Y*,sin dato,33,0-0,Marek Rodák,sin dato,1,39,7
5,6,2024-09-08,UEFA Nations League,League A,Home,Portugal,Scotland,N,sin dato,88,1-1,Angus Gunn,Nuno Mendes,1,39,7
6,7,2024-09-05,UEFA Nations League,League A,Home,Portugal,Croatia,Y*,sin dato,34,1-0,Dominik Livaković,Nuno Mendes,1,39,7
7,8,2024-08-27,Saudi Professional League,Matchweek 2,Away,Al-Nassr,Al-Fayha,Y*,sin dato,45+10,1-0,Abdulraouf Al Duqayl,sin dato,1,39,7
8,9,2024-08-22,Saudi Professional League,Matchweek 1,Home,Al-Nassr,Al-Raed,Y*,sin dato,34,0-0,Meshari Sunyur,Sadio Mané,1,39,7
9,10,2024-06-11,Friendlies (M),Friendlies (M),Home,Portugal,Rep. of Ireland,Y*,sin dato,60,2-0,Caoimhín Kelleher,Diogo Jota,1,39,7


In [36]:
#concatenamos los df que necesitamos
df_concat = pd.concat([filas_selecc,cr7], ignore_index=True)
df_concat.head(20)

,Rk,Date,Competition,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist,Goal,Edad,Dorsal
0,1,2024-10-18,Saudi Professional League,Matchweek 7,Away,Al-Nassr,Al-Shabab,Y*,sin dato,90+7,1-1,Kim Seung-gyu,sin dato,1,39,7
1,2,2024-10-12,UEFA Nations League,League A,Away,Portugal,Poland,Y*,sin dato,37,1-0,Łukasz Skorupski,sin dato,1,39,7
2,3,2024-10-05,Saudi Professional League,Matchweek 6,Home,Al-Nassr,Al-Orobah,Y*,sin dato,17,0-0,Gaëtan Coucke,sin dato,1,39,7
3,4,2024-09-27,Saudi Professional League,Matchweek 5,Home,Al-Nassr,Al-Wehda,Y*,sin dato,56,1-0,Abdullah Al-Owaishir,sin dato,1,39,7
4,5,2024-09-20,Saudi Professional League,Matchweek 4,Away,Al-Nassr,Al-Ettifaq,Y*,sin dato,33,0-0,Marek Rodák,sin dato,1,39,7
5,6,2024-09-08,UEFA Nations League,League A,Home,Portugal,Scotland,N,sin dato,88,1-1,Angus Gunn,Nuno Mendes,1,39,7
6,7,2024-09-05,UEFA Nations League,League A,Home,Portugal,Croatia,Y*,sin dato,34,1-0,Dominik Livaković,Nuno Mendes,1,39,7
7,8,2024-08-27,Saudi Professional League,Matchweek 2,Away,Al-Nassr,Al-Fayha,Y*,sin dato,45+10,1-0,Abdulraouf Al Duqayl,sin dato,1,39,7
8,9,2024-08-22,Saudi Professional League,Matchweek 1,Home,Al-Nassr,Al-Raed,Y*,sin dato,34,0-0,Meshari Sunyur,Sadio Mané,1,39,7
9,10,2024-06-11,Friendlies (M),Friendlies (M),Home,Portugal,Rep. of Ireland,Y*,sin dato,60,2-0,Caoimhín Kelleher,Diogo Jota,1,39,7


In [37]:
df_concat.loc[df_concat['Squad']=='Manchester Utd']

,Rk,Date,Competition,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist,Goal,Edad,Dorsal
86,73,2022-10-27,Europa League,Group stage,Home,Manchester Utd,Sheriff Tiraspol,Y,Left Foot,81,3-0,Maksym Koval,No assist,1,37,7
87,74,2022-10-09,Premier League,Matchweek 10,Away,Manchester Utd,Everton,N,Left Foot,44,2-1,Jordan Pickford,Casemiro,1,37,7
88,75,2022-09-15,Europa League,Group stage,Away,Manchester Utd,Sheriff Tiraspol,Y,Right Foot,39,3-0,Maksym Koval,Penalti,1,37,7
91,78,2022-05-02,Premier League,Matchweek 35,Home,Manchester Utd,Brentford,Y,Right Foot,61,3-0,David Raya,Penalti,1,37,7
92,79,2022-04-28,Premier League,Matchweek 37,Home,Manchester Utd,Chelsea,Y,Right Foot,62,1-1,Edouard Mendy,Nemanja Matic,1,37,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
893,880,2004-05-15,Premier League,Matchweek 38,Away,Manchester Utd,Aston Villa,Y,Right Foot,4,2-0,Thomas Sorensen,No assist,1,19,7
894,881,2004-04-10,Premier League,Matchweek 32,Away,Manchester Utd,Birmingham City,N,Head,60,2-1,Maik Taylor,Ryan Giggs,1,19,7
895,882,2004-03-20,Premier League,Matchweek 29,Home,Manchester Utd,Tottenham,N,Right Foot,89,3-0,Kasey Keller,Roy Keane,1,19,7
896,883,2004-02-14,FA Cup,Round 5,Home,Manchester Utd,Manchester City,Y,Right Foot,74,4-2,Ãrni Arason,No assist,1,19,7


In [38]:
#liberamos espacio
del filas_selecc

In [39]:
df_concat.shape

(904, 16)

In [40]:
#Verificamos duplicados
df_concat.duplicated().sum()

0

In [41]:
#Volvemos a llenar de manera continua la columna RK, ya que se utilizará para llenar datos faltantes
df_concat['Rk'] = range(1,len(df_concat) +1)
#verificamos el cambio
df_concat.tail(5)

,Rk,Date,Competition,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist,Goal,Edad,Dorsal
899,900,2002-11-24,Taca de Portugal,Round 4,Home,Sporting CP,CD Estarreja,N,Left Foot,67,4-1,No data,Cesar Prates,1,17,28
900,901,2002-10-26,Primeira Liga,Matchweek 8,Away,Sporting CP,Boavista,N,Right Foot,88,2-1,Carlos Martins,Tiago Ferreira,1,17,28
901,902,2002-10-07,Primeira Liga,Matchweek 6,Home,Sporting CP,Moreirense,Y,Head,90,3-0,Roberto Volpato,Rui Jorge,1,17,28
902,903,2002-10-07,Primeira Liga,Matchweek 6,Home,Sporting CP,Moreirense,Y,Right Foot,34,3-0,Roberto Volpato,No assist,1,17,28
903,904,2011-08-17,Supercopa De España,Final,Away,Real Madrid,Barcelona,Y,Right Foot,20,2-3,Víctor Valdés,Karim Benzema,1,16,7


In [42]:
#verificamos los tipos de datos
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 904 entries, 0 to 903
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Rk           904 non-null    int64         
 1   Date         904 non-null    datetime64[ns]
 2   Competition  904 non-null    object        
 3   Round        904 non-null    object        
 4   Venue        904 non-null    object        
 5   Squad        904 non-null    object        
 6   Opponent     904 non-null    object        
 7   Start        904 non-null    object        
 8   Body Part    904 non-null    object        
 9   Minute       904 non-null    object        
 10  Score        904 non-null    object        
 11  Goalkeeper   904 non-null    object        
 12  Assist       904 non-null    object        
 13  Goal         904 non-null    int64         
 14  Edad         904 non-null    int64         
 15  Dorsal       904 non-null    int64         
dtypes: datet

In [43]:
df_concat.loc[df_concat['Body Part'] == 'sin dato']

,Rk,Date,Competition,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist,Goal,Edad,Dorsal
0,1,2024-10-18,Saudi Professional League,Matchweek 7,Away,Al-Nassr,Al-Shabab,Y*,sin dato,90+7,1-1,Kim Seung-gyu,sin dato,1,39,7
1,2,2024-10-12,UEFA Nations League,League A,Away,Portugal,Poland,Y*,sin dato,37,1-0,Łukasz Skorupski,sin dato,1,39,7
2,3,2024-10-05,Saudi Professional League,Matchweek 6,Home,Al-Nassr,Al-Orobah,Y*,sin dato,17,0-0,Gaëtan Coucke,sin dato,1,39,7
3,4,2024-09-27,Saudi Professional League,Matchweek 5,Home,Al-Nassr,Al-Wehda,Y*,sin dato,56,1-0,Abdullah Al-Owaishir,sin dato,1,39,7
4,5,2024-09-20,Saudi Professional League,Matchweek 4,Away,Al-Nassr,Al-Ettifaq,Y*,sin dato,33,0-0,Marek Rodák,sin dato,1,39,7
5,6,2024-09-08,UEFA Nations League,League A,Home,Portugal,Scotland,N,sin dato,88,1-1,Angus Gunn,Nuno Mendes,1,39,7
6,7,2024-09-05,UEFA Nations League,League A,Home,Portugal,Croatia,Y*,sin dato,34,1-0,Dominik Livaković,Nuno Mendes,1,39,7
7,8,2024-08-27,Saudi Professional League,Matchweek 2,Away,Al-Nassr,Al-Fayha,Y*,sin dato,45+10,1-0,Abdulraouf Al Duqayl,sin dato,1,39,7
8,9,2024-08-22,Saudi Professional League,Matchweek 1,Home,Al-Nassr,Al-Raed,Y*,sin dato,34,0-0,Meshari Sunyur,Sadio Mané,1,39,7
9,10,2024-06-11,Friendlies (M),Friendlies (M),Home,Portugal,Rep. of Ireland,Y*,sin dato,60,2-0,Caoimhín Kelleher,Diogo Jota,1,39,7


In [44]:
#llenamos los sin datos en body part con la información 
df_concat.loc[df_concat['Date'] == '2024-10-18', ['Body Part','Assist']] = ['Right Foot', 'Penalti']
df_concat.loc[df_concat['Date'] == '2024-10-12', ['Body Part','Assist']] = ['Left Foot', 'Rafael Leao']
df_concat.loc[df_concat['Date'] == '2024-10-05', ['Body Part','Assist']] = ['Right Foot', 'Penalti']
df_concat.loc[df_concat['Date'] == '2024-09-27', ['Body Part','Assist']] = ['Right Foot', 'Penalti']
df_concat.loc[df_concat['Date'] == '2024-09-20', ['Body Part','Assist']] = ['Right Foot', 'Penalti']
df_concat.loc[df_concat['Date'] == '2024-09-08', 'Body Part'] = 'Right Foot'
df_concat.loc[df_concat['Date'] == '2024-09-05', 'Body Part'] = 'Right Foot'
df_concat.loc[df_concat['Date'] == '2024-08-27', ['Body Part','Assist']] = ['Right Foot', 'Free kick']
df_concat.loc[df_concat['Date'] == '2024-08-22', 'Body Part'] = 'Head'
df_concat.loc[(df_concat['Date'] == '2024-06-11') & (df_concat['Minute'] == '60'), 'Body Part'] = 'Left Foot'
df_concat.loc[(df_concat['Date'] == '2024-06-11') & (df_concat['Minute'] == '50'), 'Body Part'] = 'Left Foot'
df_concat.loc[(df_concat['Date'] == '2024-05-27') & (df_concat['Minute'] == '69'), ('Body Part', 'Score')] = ['Head', '2-0']
df_concat.loc[(df_concat['Date'] == '2024-05-27') & (df_concat['Minute'] == '45+3'), ('Body Part', 'Score')] = ['Left Foot', '1-0']
df_concat.loc[df_concat['Date'] == '2024-05-27', 'Goalkeeper'] = 'Al-Mahasneh'
df_concat.loc[df_concat['Date'] == '2004-02-14', 'Goalkeeper'] = 'Arni Arason'

In [45]:
df_concat.head(12)

,Rk,Date,Competition,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist,Goal,Edad,Dorsal
0,1,2024-10-18,Saudi Professional League,Matchweek 7,Away,Al-Nassr,Al-Shabab,Y*,Right Foot,90+7,1-1,Kim Seung-gyu,Penalti,1,39,7
1,2,2024-10-12,UEFA Nations League,League A,Away,Portugal,Poland,Y*,Left Foot,37,1-0,Łukasz Skorupski,Rafael Leao,1,39,7
2,3,2024-10-05,Saudi Professional League,Matchweek 6,Home,Al-Nassr,Al-Orobah,Y*,Right Foot,17,0-0,Gaëtan Coucke,Penalti,1,39,7
3,4,2024-09-27,Saudi Professional League,Matchweek 5,Home,Al-Nassr,Al-Wehda,Y*,Right Foot,56,1-0,Abdullah Al-Owaishir,Penalti,1,39,7
4,5,2024-09-20,Saudi Professional League,Matchweek 4,Away,Al-Nassr,Al-Ettifaq,Y*,Right Foot,33,0-0,Marek Rodák,Penalti,1,39,7
5,6,2024-09-08,UEFA Nations League,League A,Home,Portugal,Scotland,N,Right Foot,88,1-1,Angus Gunn,Nuno Mendes,1,39,7
6,7,2024-09-05,UEFA Nations League,League A,Home,Portugal,Croatia,Y*,Right Foot,34,1-0,Dominik Livaković,Nuno Mendes,1,39,7
7,8,2024-08-27,Saudi Professional League,Matchweek 2,Away,Al-Nassr,Al-Fayha,Y*,Right Foot,45+10,1-0,Abdulraouf Al Duqayl,Free kick,1,39,7
8,9,2024-08-22,Saudi Professional League,Matchweek 1,Home,Al-Nassr,Al-Raed,Y*,Head,34,0-0,Meshari Sunyur,Sadio Mané,1,39,7
9,10,2024-06-11,Friendlies (M),Friendlies (M),Home,Portugal,Rep. of Ireland,Y*,Left Foot,60,2-0,Caoimhín Kelleher,Diogo Jota,1,39,7


In [46]:
#buscamos la cantidad de veces que al nassr aparece de lo contrario devolverá un , esto con la finalidad de ver si los goles en este club son los correctos
df_concat['Squad'].value_counts().get('Al-Nassr',0)

70

In [47]:
df_concat.head(10)

,Rk,Date,Competition,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist,Goal,Edad,Dorsal
0,1,2024-10-18,Saudi Professional League,Matchweek 7,Away,Al-Nassr,Al-Shabab,Y*,Right Foot,90+7,1-1,Kim Seung-gyu,Penalti,1,39,7
1,2,2024-10-12,UEFA Nations League,League A,Away,Portugal,Poland,Y*,Left Foot,37,1-0,Łukasz Skorupski,Rafael Leao,1,39,7
2,3,2024-10-05,Saudi Professional League,Matchweek 6,Home,Al-Nassr,Al-Orobah,Y*,Right Foot,17,0-0,Gaëtan Coucke,Penalti,1,39,7
3,4,2024-09-27,Saudi Professional League,Matchweek 5,Home,Al-Nassr,Al-Wehda,Y*,Right Foot,56,1-0,Abdullah Al-Owaishir,Penalti,1,39,7
4,5,2024-09-20,Saudi Professional League,Matchweek 4,Away,Al-Nassr,Al-Ettifaq,Y*,Right Foot,33,0-0,Marek Rodák,Penalti,1,39,7
5,6,2024-09-08,UEFA Nations League,League A,Home,Portugal,Scotland,N,Right Foot,88,1-1,Angus Gunn,Nuno Mendes,1,39,7
6,7,2024-09-05,UEFA Nations League,League A,Home,Portugal,Croatia,Y*,Right Foot,34,1-0,Dominik Livaković,Nuno Mendes,1,39,7
7,8,2024-08-27,Saudi Professional League,Matchweek 2,Away,Al-Nassr,Al-Fayha,Y*,Right Foot,45+10,1-0,Abdulraouf Al Duqayl,Free kick,1,39,7
8,9,2024-08-22,Saudi Professional League,Matchweek 1,Home,Al-Nassr,Al-Raed,Y*,Head,34,0-0,Meshari Sunyur,Sadio Mané,1,39,7
9,10,2024-06-11,Friendlies (M),Friendlies (M),Home,Portugal,Rep. of Ireland,Y*,Left Foot,60,2-0,Caoimhín Kelleher,Diogo Jota,1,39,7


In [48]:
#agregamos filas que faltan que corresponden a goles que no estaban en la tabla que se encontró
filasnuevas_1 = pd.DataFrame({
    'Rk':[3.1],
    'Date':['2024-09-30'],
    'Competition':['AFC Champions League Elite'],
    'Round':['Group stage'],
    'Venue':['Home'],
    'Squad':['Al-Nassr'],
    'Opponent':['Al-Rayyan SC'],
    'Start':['Y*'],
    'Body Part':['Left Foot'],
    'Minute':['76'],
    'Score':['2-0'],
    'Goalkeeper':['Paulo Victor'],
    'Assist':['Abdulrahman Ghareeb'],
    'Goal':[1],
    'Edad':[39],
    'Dorsal':[7]
})

filas_nuevas2 =pd.DataFrame({
    'Rk':[9.1,9.2],
    'Date':['2024-08-17','2024-08-14'],
    'Competition':['Saudi Super Cup','Saudi Super Cup'],
    'Round':['Final', 'Semifinal'],
    'Venue':['Away', 'Away'],
    'Squad':['Al-Nassr', 'Al-Nassr'],
    'Opponent':['Al-Hilal','Al-Taawoun'],
    'Start':['Y*','Y*'],
    'Body Part':['Right Foot', 'Right Foot'],
    'Minute':['44','57'],
    'Score':['0-1', '2-0'],
    'Goalkeeper':['Bono','Mailson'],
    'Assist':['Abdulrahman Ghareeb', 'Sultan Al-Ghanam'],
    'Goal':[1,1],
    'Edad':[39,39],
    'Dorsal':[7,7]
})

In [49]:
#dividimos el df para poder concatenar más adelante en la posición correcta
df_antes = df_concat[df_concat['Rk'] <= 3]
df_entre = df_concat[(df_concat['Rk'] > 3) & (df_concat['Rk'] <= 9)]
df_desp = df_concat[df_concat['Rk'] > 9]

In [50]:
#concatenamos los dfs en la posisción correcta
df_concat = pd.concat([df_antes,filasnuevas_1,df_entre, filas_nuevas2,df_desp]).sort_values(by='Rk').reset_index(drop=True)

#verificamos el tamaño del df
df_concat.shape

(907, 16)

In [51]:
df_concat.loc[df_concat['Opponent']== 'Luxemburgo', 'Opponent'] = 'Luxembourg'

In [52]:
#liberamos memoria 
del df_antes, df_desp, df_entre, filas_nuevas2, filasnuevas_1

In [53]:
df_concat.head(13)

,Rk,Date,Competition,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist,Goal,Edad,Dorsal
0,1.0,2024-10-18 00:00:00,Saudi Professional League,Matchweek 7,Away,Al-Nassr,Al-Shabab,Y*,Right Foot,90+7,1-1,Kim Seung-gyu,Penalti,1,39,7
1,2.0,2024-10-12 00:00:00,UEFA Nations League,League A,Away,Portugal,Poland,Y*,Left Foot,37,1-0,Łukasz Skorupski,Rafael Leao,1,39,7
2,3.0,2024-10-05 00:00:00,Saudi Professional League,Matchweek 6,Home,Al-Nassr,Al-Orobah,Y*,Right Foot,17,0-0,Gaëtan Coucke,Penalti,1,39,7
3,3.1,2024-09-30,AFC Champions League Elite,Group stage,Home,Al-Nassr,Al-Rayyan SC,Y*,Left Foot,76,2-0,Paulo Victor,Abdulrahman Ghareeb,1,39,7
4,4.0,2024-09-27 00:00:00,Saudi Professional League,Matchweek 5,Home,Al-Nassr,Al-Wehda,Y*,Right Foot,56,1-0,Abdullah Al-Owaishir,Penalti,1,39,7
5,5.0,2024-09-20 00:00:00,Saudi Professional League,Matchweek 4,Away,Al-Nassr,Al-Ettifaq,Y*,Right Foot,33,0-0,Marek Rodák,Penalti,1,39,7
6,6.0,2024-09-08 00:00:00,UEFA Nations League,League A,Home,Portugal,Scotland,N,Right Foot,88,1-1,Angus Gunn,Nuno Mendes,1,39,7
7,7.0,2024-09-05 00:00:00,UEFA Nations League,League A,Home,Portugal,Croatia,Y*,Right Foot,34,1-0,Dominik Livaković,Nuno Mendes,1,39,7
8,8.0,2024-08-27 00:00:00,Saudi Professional League,Matchweek 2,Away,Al-Nassr,Al-Fayha,Y*,Right Foot,45+10,1-0,Abdulraouf Al Duqayl,Free kick,1,39,7
9,9.0,2024-08-22 00:00:00,Saudi Professional League,Matchweek 1,Home,Al-Nassr,Al-Raed,Y*,Head,34,0-0,Meshari Sunyur,Sadio Mané,1,39,7


In [54]:
df_concat['Date'] = pd.to_datetime(df_concat['Date'], errors='coerce')
df_concat.head()

,Rk,Date,Competition,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist,Goal,Edad,Dorsal
0,1.0,2024-10-18,Saudi Professional League,Matchweek 7,Away,Al-Nassr,Al-Shabab,Y*,Right Foot,90+7,1-1,Kim Seung-gyu,Penalti,1,39,7
1,2.0,2024-10-12,UEFA Nations League,League A,Away,Portugal,Poland,Y*,Left Foot,37,1-0,Łukasz Skorupski,Rafael Leao,1,39,7
2,3.0,2024-10-05,Saudi Professional League,Matchweek 6,Home,Al-Nassr,Al-Orobah,Y*,Right Foot,17,0-0,Gaëtan Coucke,Penalti,1,39,7
3,3.1,2024-09-30,AFC Champions League Elite,Group stage,Home,Al-Nassr,Al-Rayyan SC,Y*,Left Foot,76,2-0,Paulo Victor,Abdulrahman Ghareeb,1,39,7
4,4.0,2024-09-27,Saudi Professional League,Matchweek 5,Home,Al-Nassr,Al-Wehda,Y*,Right Foot,56,1-0,Abdullah Al-Owaishir,Penalti,1,39,7


In [55]:
#viendo que Rk no nos es de utilidad en este punto, lo eliminaremos y cambiaremos el nombre de la columna Edad
df_concat.drop(columns='Rk', inplace=True)
df_concat.rename(columns= {'Edad':'Age'},inplace=True)

#también cambiaremos en Assist penalti a Penalty
df_concat.loc[df_concat['Assist'] == 'Penalti', 'Assist'] = 'Penalty'
df_concat.head()

,Date,Competition,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist,Goal,Age,Dorsal
0,2024-10-18,Saudi Professional League,Matchweek 7,Away,Al-Nassr,Al-Shabab,Y*,Right Foot,90+7,1-1,Kim Seung-gyu,Penalty,1,39,7
1,2024-10-12,UEFA Nations League,League A,Away,Portugal,Poland,Y*,Left Foot,37,1-0,Łukasz Skorupski,Rafael Leao,1,39,7
2,2024-10-05,Saudi Professional League,Matchweek 6,Home,Al-Nassr,Al-Orobah,Y*,Right Foot,17,0-0,Gaëtan Coucke,Penalty,1,39,7
3,2024-09-30,AFC Champions League Elite,Group stage,Home,Al-Nassr,Al-Rayyan SC,Y*,Left Foot,76,2-0,Paulo Victor,Abdulrahman Ghareeb,1,39,7
4,2024-09-27,Saudi Professional League,Matchweek 5,Home,Al-Nassr,Al-Wehda,Y*,Right Foot,56,1-0,Abdullah Al-Owaishir,Penalty,1,39,7


In [56]:
df_concat.loc[df_concat['Competition'] == 'AFC Champions Leegue', 'Competition'] = 'AFC Champions League'

In [57]:
df_concat['Date'] = pd.to_datetime(df_concat['Date'], errors='coerce')
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 907 entries, 0 to 906
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         907 non-null    datetime64[ns]
 1   Competition  907 non-null    object        
 2   Round        907 non-null    object        
 3   Venue        907 non-null    object        
 4   Squad        907 non-null    object        
 5   Opponent     907 non-null    object        
 6   Start        907 non-null    object        
 7   Body Part    907 non-null    object        
 8   Minute       907 non-null    object        
 9   Score        907 non-null    object        
 10  Goalkeeper   907 non-null    object        
 11  Assist       907 non-null    object        
 12  Goal         907 non-null    int64         
 13  Age          907 non-null    int64         
 14  Dorsal       907 non-null    int64         
dtypes: datetime64[ns](1), int64(3), object(11)
memory usage: 

In [58]:
df_concat.loc[df_concat['Opponent'] == 'Luxembourg']

,Date,Competition,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist,Goal,Age,Dorsal
75,2023-03-26,UEFA Euro Qualifying,Group stage,Away,Portugal,Luxembourg,Y*,Left Foot,31,sin dato,Anthony Moris,Bruno Fernandes,1,38,7
76,2023-03-26,UEFA Euro Qualifying,Group stage,Away,Portugal,Luxembourg,Y*,Left Foot,9,sin dato,Anthony Moris,Nuno Mendes,1,38,7
113,2021-10-12,WCQ,First round,Home,Portugal,Luxembourg,Y*,Head,87,4-0,Anthony Moris,Rúben Neves,1,36,7
114,2021-10-12,WCQ,First round,Home,Portugal,Luxembourg,Y*,Right Foot,13,4-0,Anthony Moris,Penalty,1,36,7
115,2021-10-12,WCQ,First round,Home,Portugal,Luxembourg,Y*,Right Foot,8,4-0,Anthony Moris,Penalty,1,36,7
136,2021-03-30,WCQ,First round,Away,Portugal,Luxembourg,Y*,Right Foot,51,1-1,Anthony Moris,Joao Cancelo,1,36,7
201,2019-11-17,UEFA Euro Qualifying,Group stage,Away,Portugal,Luxembourg,Y,Right Foot,86,1-0,Anthony Moris,Diogo Jota,1,34,7
208,2019-10-11,UEFA Euro Qualifying,Group stage,Home,Portugal,Luxembourg,Y*,Right Foot,65,1-0,Anthony Moris,No assist,1,34,7
597,2012-09-07,WCQ,First round,Away,Portugal,Luxembourg,Y*,Right Foot,28,0-1,Jonathan Joubert,Joao Moutinho,1,27,7
670,2011-08-09,International Frendly,Frendly,Neutral,Portugal,Luxembourg,Y,Right Foot,26,sin dato,J. Joubert,Free kick,1,26,7


In [59]:
#limpiamos memoria
del  cr7, ronaldo

-Agregar una columna que nos de el periodo de tiempo en el que se jugó esto es

    *Primer Tiempo
    *Segundo Tiempo
    *Tiempo completo
    
-Cambiar el nombre de Dorsal por Jersey

-Agregar columna de  (Win, lose, Draw)

En PBI

-Agregar con un filtro si el gol fue en casa o fuera para ponerlo en un gráfico

-Crear medidas para el total de goles

Los DFs de Transfermarket mantendran tres columnas

    *Date
    *Match Result
    *Pos

Eliminamos las filas que en la columna Match Result son 'Uknown'


In [60]:
#cambiamos el nombre de la columna Drosal por Jersey
df_concat.rename(columns={'Dorsal':'Jersey'}, inplace=True)
df_concat.columns

Index(['Date', 'Competition', 'Round', 'Venue', 'Squad', 'Opponent', 'Start',
       'Body Part', 'Minute', 'Score', 'Goalkeeper', 'Assist', 'Goal', 'Age',
       'Jersey'],
      dtype='object')

In [61]:
#Eliminamos las filas en donde Match Result es 'Uknown'
transfermkt_df = transfermkt_df.loc[transfermkt_df['Match Result'] != 'Unknown']
transfermkt_df.head()

,Competition,Matchday,Date,Venue,For,Opponent,Result,Pos.,Minute,At score,Type of goal,Goal assist,Match Result
0,Liga Portugal,6,10/7/02,H,(12.),Moreirense(16.),3:0,LW,34',2:0,Solo run,,Win
2,Liga Portugal,8,10/26/02,A,(5.),Boavista(10.),1:2,,88',1:2,Right-footed shot,Carlos Martins,Win
3,Taça de Portugal,4.Runde,11/24/02,H,Estarreja,4:1,LW,67',3:1,,Left-footed shot,César Prates,Win
4,Taça de Portugal,5.Runde,12/18/02,H,Oliv. Hospital,8:1,SS,13',3:0,,Left-footed shot,Ricardo Fernandes,Win
5,Premier League,11,11/1/03,H,(3.),Portsmouth(11.),3:0,RW,80',2:0,Direct free kick,,Win


In [62]:
#muestra = transfermkt_df.head(10)
#procedemos a modificar las filas que están desordenadas
for index, row in transfermkt_df.iterrows():
    if pd.isna(row['At score']) or row['At score'] == '':
        transfermkt_df.at[index,'At score'] = row['Minute']
        transfermkt_df.at[index, 'Minute'] = row['Pos.']
        transfermkt_df.at[index, 'Pos.'] = row['Result']
        transfermkt_df.at[index,'Result'] = row['Opponent']
        transfermkt_df.at[index,'Opponent'] = row['For'] #movemos el valor de for a Opponent
        transfermkt_df.at[index,'For'] ='sin dato' #mantenemos sin dato el for, ya que de igual manera terminaremos ocupando solo 3 columnas

transfermkt_df.head()

,Competition,Matchday,Date,Venue,For,Opponent,Result,Pos.,Minute,At score,Type of goal,Goal assist,Match Result
0,Liga Portugal,6,10/7/02,H,(12.),Moreirense(16.),3:0,LW,34',2:0,Solo run,,Win
2,Liga Portugal,8,10/26/02,A,(5.),Boavista(10.),1:2,,88',1:2,Right-footed shot,Carlos Martins,Win
3,Taça de Portugal,4.Runde,11/24/02,H,sin dato,Estarreja,4:1,LW,67',3:1,Left-footed shot,César Prates,Win
4,Taça de Portugal,5.Runde,12/18/02,H,sin dato,Oliv. Hospital,8:1,SS,13',3:0,Left-footed shot,Ricardo Fernandes,Win
5,Premier League,11,11/1/03,H,(3.),Portsmouth(11.),3:0,RW,80',2:0,Direct free kick,,Win


In [63]:
#guardamos solo las filas que contienen más de un gol en los df de transfermkt
transfermkt_Nacionl = transfermkt_Nacionl.loc[(transfermkt_Nacionl['Goals']>= '1')]

In [64]:
#comprobamos los tipos de datos para ver que date se encuentre en los datos correctos
transfermkt_Nacionl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87 entries, 7 to 214
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Matchday             87 non-null     object
 1   Date                 87 non-null     object
 2   Venue                87 non-null     object
 3   For                  87 non-null     object
 4   Opponent             87 non-null     object
 5   Result               87 non-null     object
 6   Pos.                 87 non-null     object
 7   Goals                87 non-null     object
 8   Assists              87 non-null     object
 9   Yellow cards         87 non-null     object
 10  Second yellow cards  87 non-null     object
 11  Red cards            87 non-null     object
 12  Minutes played       87 non-null     object
 13  Match Result         87 non-null     object
dtypes: object(14)
memory usage: 10.2+ KB


In [65]:
#modificamos las fechas al formato correcto
#convertimos la columna Date al fomato Datetime, expandiendo el año
transfermkt_Nacionl['Date'] = pd.to_datetime(transfermkt_Nacionl['Date'], format='%m/%d/%y')

#cambiamos el formato al deseado yyyy/mm/dd
transfermkt_Nacionl['Date'] = transfermkt_Nacionl['Date'].dt.strftime('%Y/%m/%d')

transfermkt_Nacionl['Date'] = pd.to_datetime(transfermkt_Nacionl['Date'], errors='coerce')
#Eliminamos algunas columnas que no nos serviran 
transfermkt_Nacionl.drop(columns={'Yellow cards','Second yellow cards','Red cards','Minutes played'}, inplace=True)
transfermkt_Nacionl.head(5)

,Matchday,Date,Venue,For,Opponent,Result,Pos.,Goals,Assists,Match Result
7,Group A,2004-06-12,H,Portugal,Greece,1:2,LW,1,,Loss
11,Semi-Finals,2004-06-30,H,Portugal,Netherlands,2:1,LW,1,1,Win
13,Group 3,2004-09-04,A,Portugal,Latvia,0:2,RW,1,1,Win
14,Group 3,2004-09-08,A,Portugal,Estonia,4:0,RW,1,2,Win
16,Group 3,2004-10-13,H,Portugal,Russia,7:1,LW,2,1,Win


In [66]:
transfermkt_Nacionl.loc[transfermkt_Nacionl['Date'] == '2011-08-10']

,Matchday,Date,Venue,For,Opponent,Result,Pos.,Goals,Assists,Match Result
81,,2011-08-10,H,Portugal,Luxembourg,5:0,RW,1,1,Win


In [67]:
# la fecha anterior está mal registrada en el dfconcat, así que procedemos a cambiarla,  ya que al integrar los datos de la columna match result tendremos datos erroneos
df_concat.loc[df_concat['Date'] == '2011-08-09', 'Date'] = '2011-08-10'
df_concat.loc[df_concat['Date'] == '2011-08-10']

,Date,Competition,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist,Goal,Age,Jersey
670,2011-08-10,International Frendly,Frendly,Neutral,Portugal,Luxembourg,Y,Right Foot,26,sin dato,J. Joubert,Free kick,1,26,7


In [68]:
df_concat.loc[df_concat['Opponent'] == 'Argentina']

,Date,Competition,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist,Goal,Age,Jersey
690,2011-02-08,International Frendly,Frendly,Not say,Portugal,Argentina,Y,Right Foot,21,sin dato,S. Romero,Hugo Almeida,1,26,7


In [69]:
#ya que el dato está erroneo en ambos se procede a cambiar la fecha para un mejor uso
transfermkt_Nacionl.loc[transfermkt_Nacionl['Opponent'] == 'Argentina']

,Matchday,Date,Venue,For,Opponent,Result,Pos.,Goals,Assists,Match Result
79,,2011-02-09,A,Portugal,Argentina,1:2,LW,1,,Loss


In [70]:
df_concat.loc[df_concat['Date'] == '2011-02-08', 'Date'] = '2011-02-09'
#verificamos el cambio 
df_concat.loc[df_concat['Date'] == '2011-02-09']

,Date,Competition,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist,Goal,Age,Jersey
690,2011-02-09,International Frendly,Frendly,Not say,Portugal,Argentina,Y,Right Foot,21,sin dato,S. Romero,Hugo Almeida,1,26,7


Cambiaremos la jersey en portugal de los años 2004 a 2008,por la 17. Las fechas a utilizar están entre el 12/06/2004 al 3/04/2007 


In [71]:
#función para hacer el cambio en jersey
def cambio_jersey(df, squad, start_date, end_date, jersey):
    #nos aseguramos que la fecha está en tipo DateTime
    df['Date'] = pd.to_datetime(df['Date'])

    #convertimos las fechas inicio y fin a formato datetime
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    #filtramos y actualizamos la columna Jersey 
    df.loc[(df['Squad'] == squad) & (df['Date'] >= start_date) & (df['Date'] <= end_date), 'Jersey'] = jersey
    
    #retornamos el Dataframe
    return df

#llamamos a la función
df_concat = cambio_jersey(df_concat,'Portugal','2004-06-12','2007-04-03',17)

In [72]:
#verificamos el cambio 
df_concat.loc[df_concat['Jersey'] == 17]

,Date,Competition,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist,Goal,Age,Jersey
839,2007-03-24,UEFA Euro Qualifying,Group stage,Home,Portugal,Belgium,Y,Left Foot,75,4-0,Stijn Stijnen,No assist,1,22,17
840,2007-03-24,UEFA Euro Qualifying,Group stage,Home,Portugal,Belgium,Y,Head,55,3-0,Stijn Stijnen,Ricardo Quaresma,1,22,17
855,2006-11-15,UEFA Euro Qualifying,Group stage,Home,Portugal,Kazakhstan,Y,Right Foot,30,1-0,David Loria,No assist,1,21,17
858,2006-10-07,UEFA Euro Qualifying,Group stage,Home,Portugal,Azerbaijan,Y,Head,63,3-0,Farhad Valiyev,Simao,1,21,17
859,2006-10-07,UEFA Euro Qualifying,Group stage,Home,Portugal,Azerbaijan,Y,Left Foot,24,2-0,Farhad Valiyev,Deco,1,21,17
862,2006-06-17,World Cup,Group stage,Neutral,Portugal,Iran,Y,Right Foot,80,1-0,Ebrahim Mirzapour,Penalty,1,21,17
865,2006-03-01,International Frendly,Frendly,Neutral,Portugal,Arabia SaudÃ­,Y,Head,30,1-0,Mohamed Al Deayea,Luis Figo,1,21,17
866,2006-03-01,International Frendly,Frendly,Neutral,Portugal,Arabia SaudÃ­,Y,Left Foot,86,3-0,Mohamed Al Deayea,Hugo Viana,1,21,17
877,2005-06-08,WCQ,First round,Away,Portugal,Estonia,Y,Head,32,0-0,Artur Kotenko,Luis Figo,1,20,17
878,2005-06-04,WCQ,First round,Home,Portugal,Slovakia,Y,Right Foot,42,1-0,Kamil Contofalsky,No assist,1,20,17


In [73]:
#verificamos el df de transfermkt
transfermkt_df.head()

,Competition,Matchday,Date,Venue,For,Opponent,Result,Pos.,Minute,At score,Type of goal,Goal assist,Match Result
0,Liga Portugal,6,10/7/02,H,(12.),Moreirense(16.),3:0,LW,34',2:0,Solo run,,Win
2,Liga Portugal,8,10/26/02,A,(5.),Boavista(10.),1:2,,88',1:2,Right-footed shot,Carlos Martins,Win
3,Taça de Portugal,4.Runde,11/24/02,H,sin dato,Estarreja,4:1,LW,67',3:1,Left-footed shot,César Prates,Win
4,Taça de Portugal,5.Runde,12/18/02,H,sin dato,Oliv. Hospital,8:1,SS,13',3:0,Left-footed shot,Ricardo Fernandes,Win
5,Premier League,11,11/1/03,H,(3.),Portsmouth(11.),3:0,RW,80',2:0,Direct free kick,,Win


In [74]:
transfermkt_df.loc[transfermkt_df['Date']== '2:2'] #este es un error que se debe eliminar ya que no nos será de utilidad los datos en él
transfermkt_df = transfermkt_df[transfermkt_df['Date'] != '2:2'] #eliminamos las filas que tengan este dato 

In [75]:
transfermkt_df.loc[transfermkt_df['Result'] == ''] #este igual se eliminará
transfermkt_df = transfermkt_df[transfermkt_df['Date'] !='3:3']
transfermkt_df.shape

(510, 13)

In [76]:
#procedemos a darle formato datetime al df transfermkt_df

#convertimos la columna Date al fomato Datetime, expandiendo el año
transfermkt_df['Date'] = pd.to_datetime(transfermkt_df['Date'], format='%m/%d/%y')

#cambiamos el formato al deseado yyyy/mm/dd
transfermkt_df['Date'] = transfermkt_df['Date'].dt.strftime('%Y/%m/%d')

transfermkt_df['Date'] = pd.to_datetime(transfermkt_df['Date'], errors='coerce')
#Verificamos que se conviertan al formato correcto
transfermkt_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 510 entries, 0 to 767
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Competition   510 non-null    object        
 1   Matchday      510 non-null    object        
 2   Date          510 non-null    datetime64[ns]
 3   Venue         510 non-null    object        
 4   For           510 non-null    object        
 5   Opponent      510 non-null    object        
 6   Result        510 non-null    object        
 7   Pos.          510 non-null    object        
 8   Minute        510 non-null    object        
 9   At score      510 non-null    object        
 10  Type of goal  510 non-null    object        
 11  Goal assist   510 non-null    object        
 12  Match Result  510 non-null    object        
dtypes: datetime64[ns](1), object(12)
memory usage: 55.8+ KB


In [77]:
transfermkt_df.loc[transfermkt_df['Date'] == '2007-02-03']

,Competition,Matchday,Date,Venue,For,Opponent,Result,Pos.,Minute,At score,Type of goal,Goal assist,Match Result
46,Premier League,26,2007-02-03,A,(1.),Tottenham(8.),0:4,RW,45',0:1,Penalty,,Win


In [78]:
#hacemos el cambio de fechas para coincidir de manera correcta dónde corresponde
df_concat.loc[df_concat['Date'] == '2007-02-04','Date'] = '2007-02-03'
#verificamos el cambio 
df_concat.loc[df_concat['Date'] == '2007-02-03']

,Date,Competition,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist,Goal,Age,Jersey
844,2007-02-03,Premier League,Matchweek 26,Away,Manchester Utd,Tottenham,Y,Right Foot,45,4-0,Paul Robinson,Penalty,1,22,7


In [79]:
transfermkt_df.loc[transfermkt_df['Date'] == '2008-01-27']

,Competition,Matchday,Date,Venue,For,Opponent,Result,Pos.,Minute,At score,Type of goal,Goal assist,Match Result
78,FA Cup,4.Runde,2008-01-27,H,sin dato,Tottenham,3:1,RW,69',2:1,Penalty,,Win


In [80]:
df_concat.loc[df_concat['Date'] == '2008-01-28', 'Date'] = '2008-01-27'

In [81]:
#eliminamos las columnas que no necesitamos en los df de transfermrkt
transfermkt_Nacionl.drop(columns={'Matchday','Venue', 'For', 'Opponent', 'Result',
       'Goals', 'Assists'}, inplace=True)
transfermkt_df.drop(columns={'Competition', 'Matchday','Venue', 'For', 'Opponent', 'Result', 
                             'Minute', 'At score', 'Type of goal', 'Goal assist'}, inplace=True)

In [93]:
#considerar el outer join para realizar una columna "Result" (Win, Lose o Draw)
#outer = pd.merge(df1, df2, how="outer", on="id")
#hacemos join de los dos primeros df y posterior a eso el tercero
outer_df = pd.merge(df_concat,transfermkt_df, on='Date', how='outer')
outer_df = pd.merge(outer_df,transfermkt_Nacionl, on='Date', how='outer')
outer_df.head()

,Date,Competition,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist,Goal,Age,Jersey,Pos._x,Match Result_x,Pos._y,Match Result_y
0,2002-10-07,Primeira Liga,Matchweek 6,Home,Sporting CP,Moreirense,Y,Head,90,3-0,Roberto Volpato,Rui Jorge,1,17,28,LW,Win,NaN,NaN
1,2002-10-07,Primeira Liga,Matchweek 6,Home,Sporting CP,Moreirense,Y,Right Foot,34,3-0,Roberto Volpato,No assist,1,17,28,LW,Win,NaN,NaN
2,2002-10-26,Primeira Liga,Matchweek 8,Away,Sporting CP,Boavista,N,Right Foot,88,2-1,Carlos Martins,Tiago Ferreira,1,17,28,,Win,NaN,NaN
3,2002-11-24,Taca de Portugal,Round 4,Home,Sporting CP,CD Estarreja,N,Left Foot,67,4-1,No data,Cesar Prates,1,17,28,LW,Win,NaN,NaN
4,2002-12-18,Taca de Portugal,Round 5,Home,Sporting CP,FC Oliveira do Hospital,Y,Left Foot,13,8-1,No data,No assist,1,17,28,SS,Win,NaN,NaN


In [96]:
#prueba = outer_df.sample(frac=1).head(50)

In [94]:
#sobreescribinos los valores de match_result_x con los de Matchresult_y cuando match_result_y no es NaN

outer_df['Pos._x'] =outer_df['Pos._x'].fillna(outer_df['Pos._y'])
outer_df = outer_df.drop(columns=['Pos._y'])
outer_df.rename(columns={'Pos._x': 'Pos.'},inplace=True)

outer_df['Match Result_x']= outer_df['Match Result_x'].fillna(outer_df['Match Result_y'])
#se elimina la columna 'Match_result_y' ya que no será necesaria
outer_df = outer_df.drop(columns=['Match Result_y'])
outer_df.rename(columns={'Match Result_x':'Match Result'}, inplace=True)


In [95]:
outer_df.head()

,Date,Competition,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist,Goal,Age,Jersey,Pos.,Match Result
0,2002-10-07,Primeira Liga,Matchweek 6,Home,Sporting CP,Moreirense,Y,Head,90,3-0,Roberto Volpato,Rui Jorge,1,17,28,LW,Win
1,2002-10-07,Primeira Liga,Matchweek 6,Home,Sporting CP,Moreirense,Y,Right Foot,34,3-0,Roberto Volpato,No assist,1,17,28,LW,Win
2,2002-10-26,Primeira Liga,Matchweek 8,Away,Sporting CP,Boavista,N,Right Foot,88,2-1,Carlos Martins,Tiago Ferreira,1,17,28,,Win
3,2002-11-24,Taca de Portugal,Round 4,Home,Sporting CP,CD Estarreja,N,Left Foot,67,4-1,No data,Cesar Prates,1,17,28,LW,Win
4,2002-12-18,Taca de Portugal,Round 5,Home,Sporting CP,FC Oliveira do Hospital,Y,Left Foot,13,8-1,No data,No assist,1,17,28,SS,Win


In [96]:
#ordenaremos el Df por la columna Date de forma descendente
outer_df.sort_values(by= 'Date', ascending=False).reset_index(drop=True)
#comprobamos el cambio
outer_df.head()


,Date,Competition,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist,Goal,Age,Jersey,Pos.,Match Result
0,2002-10-07,Primeira Liga,Matchweek 6,Home,Sporting CP,Moreirense,Y,Head,90,3-0,Roberto Volpato,Rui Jorge,1,17,28,LW,Win
1,2002-10-07,Primeira Liga,Matchweek 6,Home,Sporting CP,Moreirense,Y,Right Foot,34,3-0,Roberto Volpato,No assist,1,17,28,LW,Win
2,2002-10-26,Primeira Liga,Matchweek 8,Away,Sporting CP,Boavista,N,Right Foot,88,2-1,Carlos Martins,Tiago Ferreira,1,17,28,,Win
3,2002-11-24,Taca de Portugal,Round 4,Home,Sporting CP,CD Estarreja,N,Left Foot,67,4-1,No data,Cesar Prates,1,17,28,LW,Win
4,2002-12-18,Taca de Portugal,Round 5,Home,Sporting CP,FC Oliveira do Hospital,Y,Left Foot,13,8-1,No data,No assist,1,17,28,SS,Win


In [97]:
outer_df.shape

(907, 17)

In [98]:
outer_df.isna().sum()

Date            0
Competition     0
Round           0
Venue           0
Squad           0
Opponent        0
Start           0
Body Part       0
Minute          0
Score           0
Goalkeeper      0
Assist          0
Goal            0
Age             0
Jersey          0
Pos.            6
Match Result    6
dtype: int64

In [99]:
outer_df.loc[outer_df['Competition']== 'Pro League','Competition'] = 'Saudi Professional League'

In [100]:
#guardaremos los nan con este filtro para modificarlos en un csv con mayor facilidad
valoresNan = outer_df[outer_df['Match Result'].isna()]

In [101]:
valoresNan.head(10) #verificamos en dónde se encuentran los valores faltantes para llenarlos manualmente con datos de otras paginas

,Date,Competition,Round,Venue,Squad,Opponent,Start,Body Part,Minute,Score,Goalkeeper,Assist,Goal,Age,Jersey,Pos.,Match Result
838,2023-07-31,Arab Club Champions Cup,Group stage,Neutral,Al-Nassr,US Monastir,Y*,Head,74,4-1,Sadok Yeddes,Sultan Al-Ghanam,1,38,7,NaN,NaN
839,2023-08-03,Arab Club Champions Cup,Group stage,Neutral,Al-Nassr,Zamalek,Y*,Head,87,1-1,M. Shobhy,Ghislain Konan,1,38,7,NaN,NaN
840,2023-08-06,Arab Club Champions Cup,Quarter-finals,Neutral,Al-Nassr,Raja Casablanca,Y*,Left Foot,19,3-1,A. Zniti,Talisca,1,38,7,NaN,NaN
841,2023-08-09,Arab Club Champions Cup,Semi-finals,Neutral,Al-Nassr,Al Shorta,Y*,Right Foot,75,1-0,Ahmed Basil,Penalty,1,38,7,NaN,NaN
842,2023-08-12,Arab Club Champions Cup,Final,Neutral,Al-Nassr,Al-Hilal,Y*,Head,98,2-1,Mohamed Al-Owais,No assist,1,38,7,NaN,NaN
843,2023-08-12,Arab Club Champions Cup,Final,Neutral,Al-Nassr,Al-Hilal,Y*,Right Foot,74,2-1,Mohamed Al-Owais,Sultan Al-Ghanam,1,38,7,NaN,NaN


In [102]:
outer_df.loc[outer_df['Date'] == '2023-07-31', ['Pos.','Match Result']] = ['CF','Win']
outer_df.loc[outer_df['Date'] == '2023-08-03', ['Pos.','Match Result']] = ['CF','Draw']
outer_df.loc[outer_df['Date'] == '2023-08-06', ['Pos.','Match Result']] = ['CF','Win']
outer_df.loc[outer_df['Date'] == '2023-08-09', ['Pos.','Match Result']] = ['CF','Win']
outer_df.loc[outer_df['Date'] == '2023-08-12', ['Pos.','Match Result']] = ['CF','Win']


In [103]:
outer_df.isna().sum()

Date            0
Competition     0
Round           0
Venue           0
Squad           0
Opponent        0
Start           0
Body Part       0
Minute          0
Score           0
Goalkeeper      0
Assist          0
Goal            0
Age             0
Jersey          0
Pos.            0
Match Result    0
dtype: int64

In [106]:
#Guardamos los valores para procesarlos más adelante en power BI
outer_df.to_csv('D:\\RonaldoStats\\Data\\Ronaldo_processed.csv', encoding='utf-8')
outer_df.to_parquet('D:\\RonaldoStats\\Data\\Ronaldo_processed.parquet', engine='pyarrow')